<a href="https://colab.research.google.com/github/AIkikr/code-translation-pipeline/blob/main/%E8%A8%80%E8%AA%9E%E3%82%BF%E3%82%B9%E3%82%AF_3_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# python 3.10.12
!pip install -U pip
!pip install -U transformers
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U datasets
!pip install -U peft
!pip install -U trl
!pip install -U wandb
!pip install ipywidgets --upgrade

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
)
import os, torch, gc
from datasets import load_dataset
import bitsandbytes as bnb
from trl import SFTTrainer

from transformers import DataCollatorForLanguageModeling


In [3]:
from google.colab import drive, files
from getpass import getpass
import os
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# トークンを環境変数に設定
os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token

# CLIにログイン
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [4]:
from IPython.display import Javascript
#JavaScriptコードを埋め込む
keep_alive_code = """
function keepColabAlive() {
    setInterval(() => {
        document.querySelector("colab-toolbar-button#connect").click();
    }, 600000); // 10分ごとにクリック
}
keepColabAlive();
"""

# JavaScriptを実行
display(Javascript(keep_alive_code))

<IPython.core.display.Javascript object>

In [5]:
# モデルを読み込み。
# llm-jp-3 1.8B, 3.7B, 13Bのsnapshotをダウンロード済みでmodelsディレクトリに格納してあります。
# base_model_idの値はomnicampusの環境におけるモデルのパスを表しており、それ以外の環境で実行する場合は変更の必要があります。
# その他のモデルは取得に承諾が必要なため、各自でダウンロードお願いします。
base_model_id = "microsoft/phi-4" #Fine-Tuningするベースモデル
# omnicampus以外の環境をご利用の方は以下をご利用ください。
# base_model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "phi-4-4bit-Q-lora-kura-lang" #Fine-Tuningしたモデルにつけたい名前

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# チェックポイント保存用のディレクトリを作成
!mkdir -p /content/drive/MyDrive/code_translation_project/checkpoints

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Google Driveからデータセットをダウンロード
!pip install gdown
!gdown 1xw6Edqf_nknKoei_LC49n4EtvNQezKGe

# ダウンロードしたファイルを解凍
!unzip -q codetransocean_datasets.zip -d ./data
!ls -la

Downloading...
From (original): https://drive.google.com/uc?id=1xw6Edqf_nknKoei_LC49n4EtvNQezKGe
From (redirected): https://drive.google.com/uc?id=1xw6Edqf_nknKoei_LC49n4EtvNQezKGe&confirm=t&uuid=5bdc939b-b514-4de5-9368-2b0611e72d6e
To: /content/CodeTransOcean Datasets.zip
100% 132M/132M [00:02<00:00, 58.2MB/s]
unzip:  cannot find or open codetransocean_datasets.zip, codetransocean_datasets.zip.zip or codetransocean_datasets.zip.ZIP.
total 128884
drwxr-xr-x 1 root root      4096 Mar 20 22:34  .
drwxr-xr-x 1 root root      4096 Mar 20 12:05  ..
drwxr-xr-x 6 root root      4096 Oct 23  2023 'CodeTrans Datasets'
drwxr-xr-x 8 root root      4096 Mar 20 12:15  CodeTransOcean
-rw-r--r-- 1 root root 131942209 Oct 24  2023 'CodeTransOcean Datasets.zip'
drwxr-xr-x 4 root root      4096 Mar 17 13:31  .config
drwx------ 6 root root      4096 Mar 20 12:15  drive
drwxr-xr-x 3 root root      4096 Mar 20 12:23  __MACOSX
drwxr-xr-x 1 root root      4096 Mar 17 13:32  sample_data


In [7]:
# ダウンロードしたZIPファイルを解凍
!unzip -q "CodeTransOcean Datasets.zip"

# 解凍後のディレクトリとファイルを確認
!ls -la

replace __MACOSX/._CodeTrans Datasets? [y]es, [n]o, [A]ll, [N]one, [r]ename: total 128884
drwxr-xr-x 1 root root      4096 Mar 20 22:34  .
drwxr-xr-x 1 root root      4096 Mar 20 12:05  ..
drwxr-xr-x 6 root root      4096 Oct 23  2023 'CodeTrans Datasets'
drwxr-xr-x 8 root root      4096 Mar 20 12:15  CodeTransOcean
-rw-r--r-- 1 root root 131942209 Oct 24  2023 'CodeTransOcean Datasets.zip'
drwxr-xr-x 4 root root      4096 Mar 17 13:31  .config
drwx------ 6 root root      4096 Mar 20 12:15  drive
drwxr-xr-x 3 root root      4096 Mar 20 12:23  __MACOSX
drwxr-xr-x 1 root root      4096 Mar 17 13:32  sample_data


In [5]:
# データセットディレクトリの内容を確認
!ls -la "CodeTrans Datasets"

total 36
drwxr-xr-x 6 root root 4096 Oct 23  2023 .
drwxr-xr-x 1 root root 4096 Mar 20 22:34 ..
drwxr-xr-x 2 root root 4096 Oct 23  2023 DLTrans
-rw-rw-r-- 1 root root 8196 Oct 23  2023 .DS_Store
drwxrwxr-x 2 root root 4096 Aug 28  2023 LLMTrans
drwxr-xr-x 2 root root 4096 Oct 23  2023 MultilingualTrans
drwxr-xr-x 2 root root 4096 Oct 23  2023 NicheTrans


In [6]:
# MultilingualTransディレクトリの内容を確認
!ls -la "CodeTrans Datasets/MultilingualTrans"

total 151188
drwxr-xr-x 2 root root     4096 Oct 23  2023 .
drwxr-xr-x 6 root root     4096 Oct 23  2023 ..
-rw-rw-r-- 1 root root 44466928 May 28  2023 multilingual_test.json
-rw-rw-r-- 1 root root 91702782 May 28  2023 multilingual_train.json
-rw-rw-r-- 1 root root 18623972 May 28  2023 multilingual_valid.json


In [7]:
# ファイルの最初の数行を読み込み、構造を確認
import json

# トレーニングファイルの構造を確認
train_file = "CodeTrans Datasets/MultilingualTrans/multilingual_train.json"
with open(train_file, 'r', encoding='utf-8') as file:
    first_line = file.readline().strip()
    item = json.loads(first_line)
    print("ファイル内の最初のエントリのキー:", list(item.keys()))
    print("言語キーの例：", [key for key in item.keys() if key not in ["id", "name"]])

    # 言語サンプルを表示（最初のいくつかの文字）
    for lang_key in [key for key in item.keys() if key not in ["id", "name"]]:
        print(f"\n{lang_key} サンプル (先頭100文字):")
        print(item[lang_key][:100])

ファイル内の最初のエントリのキー: ['id', 'name', 'C', 'C#']
言語キーの例： ['C', 'C#']

C サンプル (先頭100文字):
#include<graphics.h>
#include<stdlib.h>
#include<math.h>

typedef struct{
	double x,y;
}complex;

co

C# サンプル (先頭100文字):
using System.Drawing;



using System.Linq;

namespace RosettaJuliaSet
{
    class Program
    {
   


In [8]:
# ファイル全体のすべての言語キーを収集
def collect_all_language_keys(file_path):
    all_keys = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                # id や name のような非言語キーを除外
                keys = [key for key in item.keys() if key not in ["id", "name"]]
                all_keys.update(keys)
            except json.JSONDecodeError:
                continue
    return sorted(list(all_keys))

# すべての言語キーを収集
all_language_keys = collect_all_language_keys(train_file)
print("データセット内のすべての言語キー:", all_language_keys)

# すべての可能な言語ペアを作成
all_pairs = []
for i, source in enumerate(all_language_keys):
    for target in all_language_keys:
        if source != target:
            all_pairs.append((source, target))

print("\n生成した言語ペア (最初の10ペア):", all_pairs[:10])
print(f"合計言語ペア数: {len(all_pairs)}")

データセット内のすべての言語キー: ['C', 'C#', 'C++', 'Go', 'Java', 'PHP', 'Python', 'VB']

生成した言語ペア (最初の10ペア): [('C', 'C#'), ('C', 'C++'), ('C', 'Go'), ('C', 'Java'), ('C', 'PHP'), ('C', 'Python'), ('C', 'VB'), ('C#', 'C'), ('C#', 'C++'), ('C#', 'Go')]
合計言語ペア数: 56


In [9]:
# JSONLファイルからデータセットを読み込む関数
def load_jsonl_dataset(file_path, lang_pairs):
    """JSONLファイルからデータセットを読み込む関数"""
    import json
    import pandas as pd
    from datasets import Dataset, DatasetDict

    all_data = {f"{source}_to_{target}": [] for source, target in lang_pairs}

    # ファイルを行ごとに読み込む
    line_count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                line_count += 1

                # 各言語ペアに対してデータを収集
                for source, target in lang_pairs:
                    if source in item and target in item:
                        all_data[f"{source}_to_{target}"].append({
                            "source_code": item[source],
                            "target_code": item[target],
                            "source_lang": source,
                            "target_lang": target
                        })
            except json.JSONDecodeError:
                print(f"JSONデコードエラー: {line[:50]}...")
                continue

    print(f"処理した行数: {line_count}")

    # DatasetDict形式に変換
    result = {}
    for pair_name, data_list in all_data.items():
        if data_list:
            result[pair_name] = Dataset.from_pandas(pd.DataFrame(data_list))
            print(f"言語ペア {pair_name} のデータを読み込みました: {len(data_list)} サンプル")

    return DatasetDict(result)

# 選択した言語ペアでデータセットを読み込む
selected_pairs = [
    ("C++", "Python"),
    ("Python", "C++"),
    ("Java", "Python"),
    ("Python", "Java"),
    ("C#", "Python"),
    ("Python", "C#")
]

# トレーニングとテストのファイルパスが正しいことを確認
train_file = "CodeTrans Datasets/MultilingualTrans/multilingual_train.json"
test_file = "CodeTrans Datasets/MultilingualTrans/multilingual_test.json"

print("\n選択した言語ペア:", selected_pairs)
train_dataset = load_jsonl_dataset(train_file, selected_pairs)
test_dataset = load_jsonl_dataset(test_file, selected_pairs)


選択した言語ペア: [('C++', 'Python'), ('Python', 'C++'), ('Java', 'Python'), ('Python', 'Java'), ('C#', 'Python'), ('Python', 'C#')]
処理した行数: 38230
言語ペア C++_to_Python のデータを読み込みました: 1684 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 1684 サンプル
言語ペア Java_to_Python のデータを読み込みました: 1676 サンプル
言語ペア Python_to_Java のデータを読み込みました: 1676 サンプル
言語ペア C#_to_Python のデータを読み込みました: 1626 サンプル
言語ペア Python_to_C# のデータを読み込みました: 1626 サンプル
処理した行数: 15090
言語ペア C++_to_Python のデータを読み込みました: 810 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 810 サンプル
言語ペア Java_to_Python のデータを読み込みました: 872 サンプル
言語ペア Python_to_Java のデータを読み込みました: 872 サンプル
言語ペア C#_to_Python のデータを読み込みました: 398 サンプル
言語ペア Python_to_C# のデータを読み込みました: 398 サンプル


In [14]:
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# モデルとトークナイザーを読み込む
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
# LoRA適用モジュールを特定する関数
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()

    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

modules = find_all_linear_names(model)

In [25]:
# LoRA設定
peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

# LoRAをモデルに適用
model = get_peft_model(model, peft_config)

# プロンプトの定義
prompt = """### Instruction
Translate from {source_lang} to {target_lang}:
{source_code}

### Translation
{target_code}"""

EOS_TOKEN = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [10]:
def format_data(examples):
    inputs = [
        prompt.format(
            source_lang=source_lang,
            target_lang=target_lang,
            source_code=source_code,
            target_code=target_code
        ) + EOS_TOKEN
        for source_code, target_code, source_lang, target_lang in zip(
            examples["source_code"],
            examples["target_code"],
            examples["source_lang"],
            examples["target_lang"]
        )
    ]
    return tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

In [16]:
from datasets import concatenate_datasets

# 統合された単一のデータセットを作成
train_dataset_all = concatenate_datasets([ds for ds in train_dataset.values()])
test_dataset_all = concatenate_datasets([ds for ds in test_dataset.values()])

# 統合後に前処理
train_dataset_processed = train_dataset_all.map(
    format_data,
    batched=True,
    remove_columns=["source_code", "target_code", "source_lang", "target_lang"]
)

test_dataset_processed = test_dataset_all.map(
    format_data,
    batched=True,
    remove_columns=["source_code", "target_code", "source_lang", "target_lang"]
)


Map:   0%|          | 0/9972 [00:00<?, ? examples/s]

NameError: name 'EOS_TOKEN' is not defined

In [21]:
# 学習設定
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/code_translation_project/checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=10,
    warmup_steps=20,
    save_steps=100,
    save_total_limit=2,
    learning_rate=2e-5,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="none"
)

In [28]:
# SFTTrainerを使用したFine-Tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    peft_config=peft_config,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
)

Converting train dataset to ChatML:   0%|          | 0/9972 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/9972 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9972 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/4160 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/4160 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/4160 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)


In [30]:
# キャッシュを無効にして学習を開始
model.config.use_cache = False
trainer.train()

Step,Training Loss
10,0.927800
20,0.889900
30,0.939400
40,0.887600
50,0.862900
60,0.804900
70,0.805500
80,0.804100
90,0.767500
100,0.823700


Step,Training Loss
10,0.927800
20,0.889900
30,0.939400
40,0.887600
50,0.862900
60,0.804900
70,0.805500
80,0.804100
90,0.767500
100,0.823700


TrainOutput(global_step=2493, training_loss=0.631967267261372, metrics={'train_runtime': 22527.1777, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.111, 'total_flos': 4.3376592052813824e+17, 'train_loss': 0.631967267261372})

In [12]:
from google.colab import drive, files
from getpass import getpass
import os
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# トークンを環境変数に設定
os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token

# CLIにログイン
#!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

In [15]:
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN')
new_model_id = "AkiK/phi-4-4bit-Q-lora-kura-lang"

model.push_to_hub(new_model_id, token=hf_token, private=True)
tokenizer.push_to_hub(new_model_id, token=hf_token, private=True)

NameError: name 'model' is not defined

In [13]:

# 出力ディレクトリをGoogleドライブに変更
training_args.output_dir = "/content/drive/MyDrive/gemma_code_translator"

NameError: name 'training_args' is not defined

In [36]:
# モデルの保存
trainer.save_model("/content/drive/MyDrive/gemma_code_translator/final_model")

In [14]:
pip install --upgrade evaluate

In [14]:
from google.colab import output
output.enable_custom_widget_manager()

In [15]:
import torch
import gc
import random
import warnings
import sacrebleu
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import output

# メモリ解放と最適化の関数
def free_memory():
    torch.cuda.empty_cache()
    gc.collect()

# 警告の非表示と初期メモリ解放
warnings.filterwarnings('ignore', category=UserWarning)
free_memory()

# Colabのウィジェットマネージャーを有効化
output.enable_custom_widget_manager()

# ベースモデル名
base_model_id = "microsoft/phi-4"

# 4ビット量子化の設定（メモリ効率化）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True  # さらなるメモリ削減
)

# デバイスマップの設定
device_map = {
    "": 0,  # すべてのレイヤーをGPU 0に
    "lm_head": 0
}

# モデルとトークナイザーの読み込み（メモリ効率化）
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side="left")

# トークナイザーの追加設定
tokenizer.pad_token = tokenizer.eos_token

# LoRAアダプターのリポジトリ名
lora_model_name = "AkiK/phi-4-4bit-Q-lora-kura-lang"

# PeftModelとしてLoRAアダプターを読み込み
try:
    model = PeftModel.from_pretrained(
        base_model,
        lora_model_name,
        is_trainable=False,
        device_map=device_map
    )
except Exception as e:
    print(f"アダプター読み込みエラー: {e}")
    model = base_model

# モデルの最終準備
model.eval()  # 評価モードに
model = model.to('cuda')
free_memory()

# BLEU計算用の関数
def calculate_bleu(predictions, references):
    references_list = [[ref] for ref in references]
    bleu = sacrebleu.corpus_bleu(predictions, references_list)
    return {"bleu": bleu.score}

# 予測と評価の関数（バッチ処理とメモリ効率化）
def generate_and_evaluate(model, tokenizer, test_data, num_samples=50, batch_size=4):
    # ランダムサンプリング
    random.seed(42)
    test_data = random.sample(test_data, min(num_samples, len(test_data)))

    predictions = []
    references = []

    # バッチ処理
    for i in range(0, len(test_data), batch_size):
        batch_data = test_data[i:i+batch_size]
        batch_prompts = [item["prompt"] for item in batch_data]

        # バッチでのトークン化
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(model.device)

        # バッチでの生成
        with torch.no_grad():
            try:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,  # トークン数削減
                    temperature=0.2,
                    do_sample=True,
                    top_p=0.95,
                    repetition_penalty=1.1,
                    num_return_sequences=1
                )
            except RuntimeError as e:
                print(f"生成中にエラー: {e}")
                free_memory()
                continue

        # バッチ処理の結果を処理
        for j, output in enumerate(outputs):
            try:
                generated_text = tokenizer.decode(output, skip_special_tokens=True)
                prompt_length = len(batch_data[j]["prompt"])
                predicted_code = generated_text[prompt_length:].strip()

                # コードブロック抽出
                code_markers = ["```python", "```cpp", "```java", "```cs", "```c#", "```"]
                for marker in code_markers:
                    if marker in predicted_code:
                        predicted_code = predicted_code.split(marker)[1].split("```")[0].strip()
                        break

                predictions.append(predicted_code)
                references.append(batch_data[j]["target"])
            except Exception as e:
                print(f"コード抽出中にエラー: {e}")

        # 各バッチ後にメモリを解放
        free_memory()

    # BLEUスコアの計算
    bleu_score = calculate_bleu(predictions, references)

    return bleu_score, predictions, references

# テストデータの準備
test_dataset_with_prompt = []

for key, dataset in test_dataset.items():
    for item in dataset:
        # プロンプトの形式を合わせる
        prompt = f"""### Instruction
Translate from {item['source_lang']} to {item['target_lang']}:
{item['source_code']}

### Translation"""

        test_dataset_with_prompt.append({
            "prompt": prompt,
            "target": item['target_code']
        })

# メイン評価処理
try:
    # テストデータでの評価
    bleu_score, predictions, references = generate_and_evaluate(
        model,
        tokenizer,
        test_dataset_with_prompt,
        num_samples=50,
        batch_size=4
    )
    print(f"BLEUスコア: {bleu_score}")

    # 予測結果の詳細表示
    for i in range(min(5, len(predictions))):
        print(f"\n--- サンプル {i+1} ---")
        print("元のコード:", test_dataset_with_prompt[i]["prompt"])
        print("\n予測されたコード:\n", predictions[i])
        print("\n正解のコード:\n", references[i])

except Exception as e:
    print(f"評価中に致命的なエラーが発生: {e}")
finally:
    free_memory()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

BLEUスコア: {'bleu': 49.67795998533604}

--- サンプル 1 ---
元のコード: ### Instruction
Translate from C++ to Python:
#include <iostream>
#include <fstream>
#include <cmath>

using namespace std;

string readFile (string path) {
    string contents;
    string line;
    ifstream inFile(path);
    while (getline (inFile, line)) {
        contents.append(line);
        contents.append("\n");
    }
    inFile.close();
    return contents;
}

double entropy (string X) {
    const int MAXCHAR = 127;
    int N = X.length();
    int count[MAXCHAR];
    double count_i;
    char ch;
    double sum = 0.0;
    for (int i = 0; i < MAXCHAR; i++) count[i] = 0;
    for (int pos = 0; pos < N; pos++) {
        ch = X[pos];
        count[(int)ch]++;
    }
    for (int n_i = 0; n_i < MAXCHAR; n_i++) {
        count_i = count[n_i];
        if (count_i > 0) sum -= count_i / N * log2(count_i / N);
    }
    return sum;
}

int main () {
    cout<<entropy(readFile("entropy.cpp"));
    return 0;
}


### Translation

予測された

トークン数をおさえたバージョン

In [16]:
import json

results = {
    'bleu_score': bleu_score['bleu'],
    'predictions': predictions,
    'references': references
}

with open('/content/drive/MyDrive/code_translation_results.json', 'w') as f:
    json.dump(results, f, indent=4)
print("結果をJSONファイルに保存しました。")

結果をJSONファイルに保存しました。
